In [4]:
# from data import info_url,posts_url,reels_url,hashtags_url
import pandas as pd
import requests

In [ ]:
# collected data from different API calls
info_data = requests.get(info, headers=headers, params=querystring).json()
posts_data = requests.get(post, headers=headers, params=querystring).json()
reels_data = requests.get(reels, headers=headers, params=querystring).json()
hashtags_data = requests.get(
    hastag, headers=headers, params=querystring).json()

# Converting the collected data into DataFrames
info_df = pd.DataFrame(info_data['data'])
posts_df = pd.DataFrame(posts_data['data'])
reels_df = pd.DataFrame(reels_data['data'])
hashtags_df = pd.DataFrame(hashtags_data['data'])

# Save the datasets if needed
info_df.to_csv("info_data.csv", index=False)
posts_df.to_csv("posts_data.csv", index=False)
reels_df.to_csv("reels_data.csv", index=False)
hashtags_df.to_csv("hashtags_data.csv", index=False)

In [ ]:
# Remove duplicates
posts_df.drop_duplicates(inplace=True)
reels_df.drop_duplicates(inplace=True)
hashtags_df.drop_duplicates(inplace=True)

In [ ]:
# Fill missing values or remove rows with missing data
posts_df.fillna('', inplace=True)
reels_df.fillna('', inplace=True)
hashtags_df.fillna('', inplace=True)

In [ ]:
import re


def clean_text(text):
    # Removing URLs and special characters
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'[^A-Za-z0-9\s]', '', text)
    text = text.lower()
    return text


posts_df['caption'] = posts_df['caption'].apply(clean_text)
reels_df['caption'] = reels_df['caption'].apply(clean_text)
hashtags_df['hashtags'] = hashtags_df['hashtags'].apply(clean_text)

In [ ]:
# Merging all datasets into a single DataFrame
final_data = pd.merge(posts_df[['caption', 'url']], reels_df[[
                      'caption', 'url']], on='url', how='outer')
final_data = pd.merge(
    final_data, hashtags_df[['hashtags', 'url']], on='url', how='outer')

# Combine captions and hashtags for a complete content-based recommendation
final_data['content'] = final_data['caption'] + " " + final_data['hashtags']

# Drop any remaining duplicates or nulls
final_data.drop_duplicates(subset='url', inplace=True)
final_data.dropna(subset=['content'], inplace=True)

In [ ]:
# optional because hamne pahle hi nikala hai
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyzer = SentimentIntensityAnalyzer()


def get_sentiment(text):
    score = analyzer.polarity_scores(text)
    return score['compound']


final_data['sentiment'] = final_data['content'].apply(get_sentiment)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Vectorizing the text content
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(final_data['content'])

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# Compute the cosine similarity matrix
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [ ]:
# Create a function to recommend based on content
def get_recommendations(index, cosine_sim=cosine_sim):
    # Get the pairwise similarity scores of all content
    sim_scores = list(enumerate(cosine_sim[index]))

    # Sort content based on similarity score
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the indices of the top 10 most similar content
    sim_scores = sim_scores[1:11]

    # Return the URLs of the most similar content
    content_indices = [i[0] for i in sim_scores]
    return final_data['url'].iloc[content_indices]

In [ ]:
# Test the recommendation system for the first post
print(get_recommendations(0))

In [ ]:
import pickle

# Save the TF-IDF vectorizer
with open('tfidf_vectorizer.pkl', 'wb') as f:
    pickle.dump(tfidf, f)

# Save the cosine similarity matrix
with open('cosine_similarity.pkl', 'wb') as f:
    pickle.dump(cosine_sim, f)

In [1]:
import pandas as pd
from faker import Faker
import random
import numpy as np

In [13]:

# Initialize Faker
fake = Faker()

# Function to create synthetic dataset


def create_synthetic_data(num_samples=1000):
    data = {
        'UserID': [],
        'Username': [],
        'Caption': [],
        'Hashtags': [],
        'Likes': [],
        'Comments': []
    }

    for _ in range(num_samples):
        # Generate user data
        user_id = random.randint(0, 1000)
        username = fake.user_name()
        caption = fake.sentence(nb_words=random.randint(5, 15))
        hashtags = ' '.join([fake.word() for _ in range(random.randint(1, 5))])
        likes = random.randint(0, 1000)
        comments = random.randint(0, 200)

        # Append data
        data['UserID'].append(user_id)
        data['Username'].append(username)
        data['Caption'].append(caption)
        data['Hashtags'].append(hashtags)
        data['Likes'].append(likes)
        data['Comments'].append(comments)

    return pd.DataFrame(data)


# Create synthetic dataset
synthetic_data = create_synthetic_data(num_samples=1000)

# Save to CSV
synthetic_data.to_csv('synthetic_instagram_data.csv', index=False)
print(synthetic_data.head())

   UserID       Username                                            Caption  \
0     463  brandonrogers  Nature reason every phone green approach artic...   
1     672       thomas48  Into team prove wide police entire wear everyb...   
2     240    longpatrick  Indeed bank cost senior treatment receive bit ...   
3     635         clynch         Movement talk need inside opportunity lot.   
4     665   michaellopez       Sit over thus ok point family range officer.   

                   Hashtags  Likes  Comments  
0       marriage bring life    402       196  
1                      draw    163       184  
2        list school strong    905        40  
3  strong money down though    356        82  
4                 play news    991        66  


In [3]:
df = pd.read_csv('synthetic_instagram_data.csv')
df.head(1000)

,UserID,Username,Caption,Hashtags,Likes,Comments
0,463,brandonrogers,Nature reason every phone green approach artic...,marriage bring life,402,196
1,672,thomas48,Into team prove wide police entire wear everyb...,draw,163,184
2,240,longpatrick,Indeed bank cost senior treatment receive bit ...,list school strong,905,40
3,635,clynch,Movement talk need inside opportunity lot.,strong money down though,356,82
4,665,michaellopez,Sit over thus ok point family range officer.,play news,991,66
...,...,...,...,...,...,...
995,72,sbrooks,Who fish same break throughout door reveal tro...,population class us animal,976,131
996,941,alibrittany,Democrat much student three hear job professor...,hour he western these remain,624,181
997,837,catherinecunningham,To consider sign garden resource natural start...,stay,244,0
998,130,amy46,She offer south week evidence along yet that r...,certain what career,785,138
